Instaliuojame reikalingas bibliotekas

In [1]:
!pip install openimages
!pip install torch torchvision torchaudio --upgrade

   ---------------------------------------- 0.0/198.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/198.6 MB 1.7 MB/s eta 0:01:54
   ---------------------------------------- 0.1/198.6 MB 1.3 MB/s eta 0:02:32
   ---------------------------------------- 0.2/198.6 MB 2.0 MB/s eta 0:01:42
   ---------------------------------------- 0.4/198.6 MB 2.2 MB/s eta 0:01:29
   ---------------------------------------- 0.5/198.6 MB 2.2 MB/s eta 0:01:31
   ---------------------------------------- 1.0/198.6 MB 3.5 MB/s eta 0:00:58
   ---------------------------------------- 1.8/198.6 MB 5.6 MB/s eta 0:00:36
    --------------------------------------- 3.4/198.6 MB 8.9 MB/s eta 0:00:22
    --------------------------------------- 4.6/198.6 MB 11.3 MB/s eta 0:00:18
   - -------------------------------------- 5.4/198.6 MB 11.6 MB/s eta 0:00:17
   - -------------------------------------- 6.9/198.6 MB 13.4 MB/s eta 0:00:15
   - -------------------------------------- 8.8/198.6 MB 15.6 MB/s e

Pasirenkame klases, kurias tirsime

In [2]:
Class1 = 'Fountain'
Class2 = 'Mushroom'
Class3 = 'Strawberry'

Parsisiunčiame nuotraukas

In [3]:
from openimages.download import download_dataset

data_dir = 'data'
classes = [Class1, Class2, Class3]
number_for_samples = 200

# download_dataset(data_dir, classes, limit=number_for_samples)

Aprašome pagrindines transformacijas, sukuriame duomenų rinkinį ir užkrauname pirmas nuotraukas (batch)

In [4]:
import torch
from torchvision import transforms, datasets

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = datasets.ImageFolder(root=data_dir, transform=preprocess)
class_to_idx = dataset.class_to_idx

batch_holder = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=True, num_workers=4)

Patikriname ar įjungtas GPU ir sukuriame modėlį, kurį naudosime nuotraukų vertinimui

In [ ]:
from torchvision import models

# if torch.cuda.is_available():
#     device = torch.device('cuda')
# else:
#     raise SystemError("CUDA is not available. This script requires a GPU.")

model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
# model.to(device)
model.eval()

Apskaičiuojame reikšmes
*  Tikslumas (Accuracy): Teisingai numatytų stebėjimų proporcija iš visų stebėjimų.
*  Atkūrimas (Recall): Tikrųjų teigiamų atvejų, kurie buvo teisingai identifikuoti, proporcija.
*   Precizija (Precision): Teigiamų identifikacijų, kurios iš tikrųjų buvo teisingos, proporcija.
*   F1 statistikos: Precizijos ir atpažinimo tikslumo svoris vidurkis.

In [9]:
import numpy as np

def calculate_metrics(ground_truth, predictions, threshold=0.5):
  predictions_thresholded = (predictions >= threshold).astype(np.float64)
  TP = np.sum((predictions_thresholded == 1) & (ground_truth == 1))
  TN = np.sum((predictions_thresholded == 0) & (ground_truth == 0))
  FP = np.sum((predictions_thresholded == 1) & (ground_truth == 0))
  FN = np.sum((predictions_thresholded == 0) & (ground_truth == 1))

  accuracy = (TP + TN) / (TP + FP + TN + FN) if (TP + FP + TN + FN) > 0 else 0
  recall = TP / (TP + FN) if (TP + FN) > 0 else 0
  precision = TP / (TP + FP) if (TP + FP) > 0 else 0
  f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

  return {
    'accuracy': accuracy,
    'recall': recall,
    'precision': precision,
    'f1': f1
  }

Ištraukiam tikimybes tam tikrai klasei ir sulyginam jas su atitinkamomis tikrosiomis žymėmis. Išvedam sąrašą porų, kiekvienoje iš jų yra tikimybė ir dvejetainė tikroji vertė, nurodanti, ar klasės žymė atitinka laukiamą klasę.

In [10]:
def calculate_prob(arr2D, class_id, label_arr, class_label):
  result = []
  label_bool = 1.0
  for i in range(0, len(label_arr)):
    if class_label == label_arr[i]:
      label_bool = 1.0
    else:
      label_bool = 0.0
    result.append((arr2D[i][class_id], label_bool))
  return result

Paimam `calculate_prob` funkcijos sugeneruotą porų sąrašą ir ištraukiam tik tikimybes į sąrašą.

In [11]:
def extract_prob(arr):
  result = []
  for prob, truth in arr:
      prob = prob.cpu().numpy() if prob.is_cuda else prob.numpy()
      result.append(prob)
  return result


Panašiai kaip `extract_prob`, bet ši funkcija ištraukia tikrasias vertes iš porų sąrašo.

In [12]:
def extract_gt(arr):
  result = []
  for prob, truth in arr:
    result.append(truth)
  return result

Ši funkcija derina ankstesnių funkcijų naudojimą, ištraukia tikrąsias žymes ir numatytas tikimybes, tada naudoja šiuos duomenis metrikoms apskaičiuoti, kviesdama `calculate_metrics`.


In [13]:
def extractAndCalculate(gt_prob_list, treshold):
  gt_list = extract_gt(gt_prob_list)
  pred_list = extract_prob(gt_prob_list)
  return calculate_metrics(np.array(gt_list), np.array(pred_list), treshold)


Ši funkcija atranda geriausią threshold kiekvienai klasei.

In [16]:
import numpy as np
from sklearn.metrics import f1_score

def find_best_threshold_for_class(probabilities, ground_truths):
    thresholds = np.arange(0.0, 1.05, 0.05)
    best_threshold = 0.5
    best_f1 = 0.0

    for threshold in thresholds:
        predictions = (probabilities >= threshold).astype(int)
        f1 = f1_score(ground_truths, predictions)

        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    return best_threshold, best_f1

Pritaikome geriausią threshold

In [17]:
def apply_threshold(probabilities, threshold):
    return (probabilities >= threshold).astype(int)

Aprašome kintamuosius ir parenkame class_id pagal klases.

In [18]:
class1_id = class_to_idx[Class1.lower()]
class2_id = class_to_idx[Class2.lower()]
class3_id = class_to_idx[Class3.lower()]

class1_probabilities = []
class2_probabilities = []
class3_probabilities = []

class1_ground_truths = []
class2_ground_truths = []
class3_ground_truths = []

Kiekvienas rinkinys (batch) (po 64 nuotraukas) siunčiamas į modelį, kuris kiekvienai nuotraukai atiduoda 1000 ilgio sąrašą: [tikimybė_kad_nuotraukoje_klasė1, ... ,  tikimybė_kad_nuotraukoje_klasė1000]

Skaičiuojami tikslumas (accuracy), atkūrimas (recall), precizija (precision) ir F1 statistikos kiekvienai klasei atskirai

In [20]:
import torch

for images, labels in batch_holder:
    # images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    probabilities = torch.sigmoid(outputs).detach()

    probabilities_np = probabilities.cpu().numpy()
    labels_np = labels.cpu().numpy()

    class1_probabilities.extend(probabilities_np[:, class1_id])
    class2_probabilities.extend(probabilities_np[:, class2_id])
    class3_probabilities.extend(probabilities_np[:, class3_id])

    num_classes = model.fc.out_features
    one_hot_labels = np.eye(num_classes)[labels_np]
    class1_ground_truths.extend(one_hot_labels[:, class1_id])
    class2_ground_truths.extend(one_hot_labels[:, class2_id])
    class3_ground_truths.extend(one_hot_labels[:, class3_id])

class1_probabilities = np.array(class1_probabilities)
class2_probabilities = np.array(class2_probabilities)
class3_probabilities = np.array(class3_probabilities)

class1_ground_truths = np.array(class1_ground_truths)
class2_ground_truths = np.array(class2_ground_truths)
class3_ground_truths = np.array(class3_ground_truths)

best_threshold_class1, best_f1_class1 = find_best_threshold_for_class(class1_probabilities, class1_ground_truths)
best_threshold_class2, best_f1_class2 = find_best_threshold_for_class(class2_probabilities, class2_ground_truths)
best_threshold_class3, best_f1_class3 = find_best_threshold_for_class(class3_probabilities, class3_ground_truths)

print(f"Best threshold for {Class1}: {best_threshold_class1} with F1: {best_f1_class1}")
print(f"Best threshold for {Class2}: {best_threshold_class2} with F1: {best_f1_class2}")
print(f"Best threshold for {Class3}: {best_threshold_class3} with F1: {best_f1_class3}")

class1_metrics = calculate_metrics(class1_ground_truths, apply_threshold(class1_probabilities, best_threshold_class1), best_threshold_class1)
class2_metrics = calculate_metrics(class2_ground_truths, apply_threshold(class2_probabilities, best_threshold_class2), best_threshold_class2)
class3_metrics = calculate_metrics(class3_ground_truths, apply_threshold(class3_probabilities, best_threshold_class3), best_threshold_class3)

print(f"{Class1} statistics with best threshold: {class1_metrics}")
print(f"{Class2} statistics with best threshold: {class2_metrics}")
print(f"{Class3} statistics with best threshold: {class3_metrics}")

Best threshold for Fountain: 0.0 with F1: 0.5
Best threshold for Mushroom: 0.0 with F1: 0.5
Best threshold for Strawberry: 0.4 with F1: 0.5195154777927322
Fountain statistics with best threshold: {'accuracy': 0.3333333333333333, 'recall': 1.0, 'precision': 0.3333333333333333, 'f1': 0.5}
Mushroom statistics with best threshold: {'accuracy': 0.3333333333333333, 'recall': 1.0, 'precision': 0.3333333333333333, 'f1': 0.5}
Strawberry statistics with best threshold: {'accuracy': 0.405, 'recall': 0.965, 'precision': 0.35543278084714547, 'f1': 0.5195154777927321}
